## Building a Regression Model using Keras

### Table of contents
1. Donwnload and clean dataset
2. Import Keras
3. Build a Neural Network
4. Train and Test the Neural Network

### Download and Clean Dataset

We will start by importing the necessary libraries for the project

In [1]:
import time
import os
import pandas as pd
import numpy as np


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [2]:
conda install -c conda-forge keras

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


The dataset that we will use is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used. The ingredients include:
1. Cement
2. Blast furnace slag
3. Fly ash
4. Water
5. Superplasticizer
6. Coarse aggregate
7. Fine aggregate

We will download the data and read it into a pandas daataframe

In [3]:
concrete_df = pd.read_csv("/Users/juan/Downloads/concrete_data.csv")
concrete_df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


The first sample consists of 540 cubic meters of cement, 0 cubic meters of blast furnace slag and fly ash, 162 cubic meters of water, 2.5 cubic meters of superplasticizer, 1040 cubic meters of fine aggregate, 676 cubic meters of fine aggregate and has a compressive strength of 79.99 MPa at 28 days.

Check how many data points we have in the data frame

In [4]:
concrete_df.shape

(1030, 9)

The next step is to check the dataset for any missing values

In [5]:
concrete_df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The dataset looks clean so we can proceed to build our model

#### Split data intro predictors and target

In [7]:
concrete_df_columns = concrete_df.columns
predictors = concrete_df[concrete_df_columns[concrete_df_columns != "Strength"]] # All columns except Strength
target = concrete_df["Strength"] #Only the column Strength

Check the first rows for the predictors and target data

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to n_cols variables, because we will need it when we build our network

In [10]:
n_cols = predictors.shape[1] #number of predictors

### Importing packages from Keras library

Let's import the packages from the Keras library that we will need to build our regression model

In [11]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Part A: Build a baseline model

Use the Keras library to build a neural network with the following:

One hidden layer of 10 nodes, and a ReLU activation function

Use the adam optimizer and the mean squared error as the loss function.

Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

Train the model on the training data using 50 epochs.

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

Report the mean and the standard deviation of the mean squared errors.

#### Build the neural network model

Let's create a function that defines our regression model and we can call it to create our model

In [12]:
#define the regression model with one hidden layer
def regression_model():
    # Create model
    model = Sequential()

    model.add(Dense(10, activation="relu", input_shape=(n_cols,)))
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

####  Train and test the model

We will create the regression model

In [13]:
model = regression_model()

Instructions for updating:
Colocations handled automatically by placer.


Randomly split the data into a training and test sets by holding 30%  of the data for testing. You can use the train_test_split helper function 

In [14]:
X = predictors
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)   
print("Training set: ", X_train.shape, y_train.shape)
print("Testing set: ", X_test.shape, y_test.shape)

Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)


Train the model on the training data using 50 epochs

In [15]:
model.fit(X_train, y_train, epochs=50, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 0s - loss: 31539.6267
Epoch 2/50
 - 0s - loss: 5951.5542
Epoch 3/50
 - 0s - loss: 3834.3502
Epoch 4/50
 - 0s - loss: 3413.5374
Epoch 5/50
 - 0s - loss: 3052.0545
Epoch 6/50
 - 0s - loss: 2736.0039
Epoch 7/50
 - 0s - loss: 2450.6898
Epoch 8/50
 - 0s - loss: 2203.1207
Epoch 9/50
 - 0s - loss: 1973.3979
Epoch 10/50
 - 0s - loss: 1771.3285
Epoch 11/50
 - 1s - loss: 1601.9896
Epoch 12/50
 - 0s - loss: 1450.0877
Epoch 13/50
 - 0s - loss: 1320.6769
Epoch 14/50
 - 0s - loss: 1208.7550
Epoch 15/50
 - 0s - loss: 1110.8977
Epoch 16/50
 - 0s - loss: 1022.0888
Epoch 17/50
 - 0s - loss: 946.9646
Epoch 18/50
 - 0s - loss: 879.9615
Epoch 19/50
 - 0s - loss: 819.2578
Epoch 20/50
 - 0s - loss: 765.9928
Epoch 21/50
 - 0s - loss: 719.1378
Epoch 22/50
 - 0s - loss: 675.4106
Epoch 23/50
 - 0s - loss: 637.7544
Epoch 24/50
 - 0s - loss: 603.0703
Epoch 25/50
 - 0s - loss: 568.5382
Epoch 26/50
 - 0s - loss: 538.3694
Epoch 27/50
 - 0s - loss: 512.2260

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.    
    

In [16]:
y_hat = model.predict(X_test)   
mse = mean_squared_error(y_test, y_hat)

In [17]:
print(mse)

189.78865346285178


Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors

We will create one function that performs steps 1 to 3 and another function that iterates 50 times and creates the list of 50 mean squared errors

In [18]:
def get_mean_squared_error(compiled_model, X, y, epochs=50, verbose=1):
    """Get report (dataframe) of two metrics: 
    The mean and the standard deviation of the mean squared errors
    """   
    
    # 1. Randomly split the data into a training and test sets by holding 30% 
    # of the data for testing. You can use the train_test_split helper function 
    # from Scikit-learn. 
    X = predictors
    y = target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)   
    print("Training set: ", X_train.shape, y_train.shape)
    print("Testing set: ", X_test.shape, y_test.shape)
    
    
    # 2. Train the model on the training data using 50 epochs.
    # Fit the built model with training set
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)    

    # 3. Evaluate the model on the test data and compute the mean squared error 
    # between the predicted concrete strength and the actual concrete strength. 
    # You can use the mean_squared_error function from Scikit-learn.    
    y_hat = model.predict(X_test)    
    mse = mean_squared_error(y_test, y_hat)
    
    # Return the mean squared error
    return mse

In [19]:
#Function to round the calculation of the mean and std deviation to 2 decimal places
def get_round(score, num_of_digits=2):
    """Get round with given number of decimal digits 
    """
    return round(score, num_of_digits)

#Function to calculate the mean of the list of mean squared errors
def get_mean(list_of_mse_scores):
    """Get mean
    """
    if list_of_mse_scores:
        return get_round(np.mean(list_of_mse_scores))
    return None

#Function to calculate the standard deviation of the list of mean squared errors
def get_standard_deviation(list_of_mse_scores):
    """Get standard deviation
    """
    if list_of_mse_scores:
        return get_round(np.std(list_of_mse_scores))
    return None




#Function to iterate and calculate the mean squared error
def get_mean_and_std_of_mse(df_X, 
                            df_y, 
                            compiled_model,                
                            max_iteration=50, 
                            epochs=50, 
                            verbose=0):
    """Generate the mean and the standard deviation of the mean squared errors 
    """
    # Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.    
    list_of_mean_squared_errors = []
    for i in range(max_iteration):
        start_time = time.time()
        print("-" * 36)
        print("Processing current number of iteration : {}".format(i+1))        
        mse = get_mean_squared_error(compiled_model, df_X, df_y, epochs=epochs, verbose=verbose)
        list_of_mean_squared_errors.append(mse)
        print("Duration (seconds): {}".format(time.time()-start_time))
    # end for

    print("Finished - {} times.\nAnd the list of mean squared errors : {}".format(max_iteration,
                                                                                  list_of_mean_squared_errors))
    
    mean_mse = get_mean(list_of_mean_squared_errors)
    std_mse = get_standard_deviation(list_of_mean_squared_errors)

    print("-" * 72)
    print("The mean and the standard deviation of the mean squared errors are: {} and {}, respectively".format(
           mean_mse, std_mse))
    
    return mean_mse, std_mse




In [20]:
max_iteration = 50
epochs = 50
verbose = 2

# Get the compiled model
model = regression_model()

mean_mse, std_mse = get_mean_and_std_of_mse(predictors, target, model, max_iteration=max_iteration, epochs=epochs, verbose=verbose)

------------------------------------
Processing current number of iteration : 1
Training set:  (721, 8) (721,)
Testing set:  (309, 8) (309,)
Epoch 1/50
 - 0s - loss: 5860.6105
Epoch 2/50
 - 0s - loss: 4648.5035
Epoch 3/50
 - 0s - loss: 3643.2372
Epoch 4/50
 - 0s - loss: 2862.3066
Epoch 5/50
 - 0s - loss: 2271.9930
Epoch 6/50
 - 0s - loss: 1813.3072
Epoch 7/50
 - 0s - loss: 1463.8464
Epoch 8/50
 - 0s - loss: 1182.8254
Epoch 9/50
 - 0s - loss: 975.8607
Epoch 10/50
 - 0s - loss: 800.9978
Epoch 11/50
 - 0s - loss: 680.5277
Epoch 12/50
 - 0s - loss: 570.6739
Epoch 13/50
 - 0s - loss: 495.1797
Epoch 14/50
 - 0s - loss: 432.6074
Epoch 15/50
 - 0s - loss: 379.5628
Epoch 16/50
 - 0s - loss: 342.6261
Epoch 17/50
 - 0s - loss: 309.7410
Epoch 18/50
 - 0s - loss: 285.6126
Epoch 19/50
 - 0s - loss: 270.8148
Epoch 20/50
 - 0s - loss: 252.7110
Epoch 21/50
 - 0s - loss: 234.0699
Epoch 22/50
 - 0s - loss: 226.6116
Epoch 23/50
 - 0s - loss: 214.4260
Epoch 24/50
 - 0s - loss: 203.5099
Epoch 25/50
 - 0s - 

### Report the mean and standard deviation of the mean squared error

The mean and standard deviation of the mean squared error after 50 iterations, for the case of not normalized data is:

In [21]:
def get_report(name_of_case, mean_mse, std_mse):
    """Get report of mse and std: 
    The mean and the standard deviation of the mean squared errors
    """
    COL_NAME_EXPERIMENT = "Experiment"
    COL_NAME_MSE = "Mean MSE"
    COL_NAME_RMSE = "Std Deviation MSE"
    header_of_mse_and_rmse = [COL_NAME_EXPERIMENT, COL_NAME_MSE, COL_NAME_RMSE]
    values = [[name_of_case, mean_mse, std_mse]]

    return pd.DataFrame(columns=header_of_mse_and_rmse, data=values)

In [22]:
name_of_case = "Baseline-not normalized (50 epochs)"

# Report the mean and the standard deviation of the mean squared errors
df_baseline = get_report(name_of_case, mean_mse, std_mse)
df_baseline

,Experiment,Mean MSE,Std Deviation MSE
0,Baseline-not normalized (50 epochs),61.82,22.41
